# Import required libraries

In [1]:
# pip install seaborn
# pip install xgboost
# pip install keras
# pip install sklearn
# pip install pandas
# pip install numpy

In [2]:
import pandas as pd

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


# Import the data

In [4]:
# help(pd.read_excel)

In [5]:
# df_x = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet1')

In [6]:
df_x = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet1')

In [7]:
df_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df_x.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21'],
      dtype='object')

In [9]:
# df_y = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet2')

In [10]:
df_y = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet2')

In [11]:
df_y.head(2)

,Column1
0,66.193
1,66.193


In [12]:
df_y = df_y.rename(columns={'Column1': 'Position'})

In [13]:
df_y.head(2)

,Position
0,66.193
1,66.193


In [14]:
df_all = df_x
df_all['Position'] = df_y['Position']

In [15]:
df_all.to_csv('all-combined-data.csv', index = False)

In [16]:
len(df_x)

729633

In [17]:
train_x = df_x[0:600000]
test_x = df_x[600000:]
train_y = df_y[0:600000]
test_y = df_y[600000:]

In [18]:
# df_all.head(2)

In [19]:
# from sklearn.model_selection import train_test_split

# train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [20]:
train_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Position
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66.193
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,66.193


In [21]:
len(test_x)

129633

In [22]:
train_x.shape[1]

22

# Trying a Deep Neural Network

#### Followed this blogpost: https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

In [23]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               2944      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 167,809
Trainable params: 167,809
Non-trainable params: 0
_________________________________________________________________


In [24]:
checkpoint_name = 'models/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [25]:
callbacks_list

In [26]:
train = train_x.to_numpy()

In [27]:
train.shape

(600000, 22)

In [28]:
target = train_y.to_numpy()

In [29]:
target.shape

(600000, 1)

In [30]:
NN_model.fit(train, target, epochs=50, batch_size=64, validation_split = 0.2, callbacks=callbacks_list)

Train on 480000 samples, validate on 120000 samples
Epoch 1/50
480000/480000 [==============================] - 47s 98us/step - loss: 3.4014 - mean_absolute_error: 3.4014 - val_loss: 0.7946 - val_mean_absolute_error: 0.7946

Epoch 00001: val_loss improved from inf to 0.79459, saving model to models/Weights-001--0.79459.hdf5
Epoch 2/50
480000/480000 [==============================] - 45s 94us/step - loss: 2.0663 - mean_absolute_error: 2.0663 - val_loss: 0.5631 - val_mean_absolute_error: 0.5631

Epoch 00002: val_loss improved from 0.79459 to 0.56310, saving model to models/Weights-002--0.56310.hdf5
Epoch 3/50
480000/480000 [==============================] - 44s 92us/step - loss: 1.6115 - mean_absolute_error: 1.6115 - val_loss: 2.6539 - val_mean_absolute_error: 2.6539

Epoch 00003: val_loss did not improve from 0.56310
Epoch 4/50
480000/480000 [==============================] - 45s 94us/step - loss: 1.4118 - mean_absolute_error: 1.4118 - val_loss: 3.2473 - val_mean_absolute_error: 3.2473



Epoch 00035: val_loss did not improve from 0.04421
Epoch 36/50
480000/480000 [==============================] - 46s 96us/step - loss: 0.5850 - mean_absolute_error: 0.5850 - val_loss: 0.5537 - val_mean_absolute_error: 0.5537

Epoch 00036: val_loss did not improve from 0.04421
Epoch 37/50
480000/480000 [==============================] - 44s 91us/step - loss: 0.6014 - mean_absolute_error: 0.6014 - val_loss: 0.9613 - val_mean_absolute_error: 0.9613

Epoch 00037: val_loss did not improve from 0.04421
Epoch 38/50
480000/480000 [==============================] - 48s 100us/step - loss: 0.5679 - mean_absolute_error: 0.5679 - val_loss: 0.5259 - val_mean_absolute_error: 0.5259

Epoch 00038: val_loss did not improve from 0.04421
Epoch 39/50
480000/480000 [==============================] - 42s 87us/step - loss: 0.5718 - mean_absolute_error: 0.5718 - val_loss: 1.1866 - val_mean_absolute_error: 1.1866

Epoch 00039: val_loss did not improve from 0.04421
Epoch 40/50
480000/480000 [====================

In [35]:
wights_file = 'models/Weights-033--0.04421.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [36]:
test = test_x.to_numpy()

In [37]:
predictions = NN_model.predict(test)

In [38]:
predictions[1]

array([215.47728], dtype=float32)

In [39]:
actual = test_y.to_numpy()http://localhost:8888/notebooks/deep%20learning/yohan%20(2).ipynb#We-will-judge-our-model-on-Mean-Absolute-Error-Metric

In [40]:
actual[1]

array([215.35])

### We will judge our model on Mean Absolute Error Metric

In [41]:
MAE = mean_absolute_error(actual , predictions)
print('Deep Neural Network validation MAE = ', MAE)

Deep Neural Network validation MAE =  0.041667498268297826


# Trying CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
# CNN Layers
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])